In [54]:
import requests
import pandas as pd
import numpy as np
import glob
import os
import re
import time
import pandas as pd
from apify_client import ApifyClient
from bs4 import BeautifulSoup
import re
import json
from requests.exceptions import HTTPError, ConnectionError
from urllib3.exceptions import ProtocolError
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI


### Getting Master DB Data

In [55]:

# --- CONFIG ---
file_path = "./Master DB/Master_DB_oct22.xlsx"

# --- HELPER FUNCTIONS ---
def clean_uen(u: str) -> str | None:
    if pd.isna(u):
        return None
    return re.sub(r"[^A-Z0-9]", "", str(u).upper().strip())

def clean_text(text: str) -> str | None:
    if pd.isna(text):
        return None
    text = str(text).strip().upper()
    return None if text == "NAN" else text

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert all column names to uppercase, replace non-alphanumeric with single underscore, remove trailing underscores."""
    new_cols = []
    for col in df.columns:
        col_std = re.sub(r"[^A-Z0-9]", "_", col.upper().strip())
        col_std = re.sub(r"_+", "_", col_std)  # Replace multiple underscores with single
        col_std = col_std.strip("_")  # Remove leading/trailing underscores
        new_cols.append(col_std)
    df.columns = new_cols
    return df

# --- LOAD DATA ---
master_db_df = pd.read_excel(file_path)

# --- SELECT RELEVANT COLUMNS ---
columns_to_keep = [
    "Company Registration Number (UEN)",
    "ACRA REGISTERED NAME",
    "Brand/Deal Name/Business Name",
    "Primary SSIC Code",
    "PIC NAME 1 Contact Number",
    "PIC 1 email address",
    "Website URL",
    "Parent Industry Type",
    "Sub Industry"
]
master_db_df = master_db_df[columns_to_keep].copy()

# --- STANDARDIZE COLUMN NAMES ---
master_db_df = standardize_columns(master_db_df)

# --- CLEANING & RENAME SPECIFIC COLUMNS ---
# Dynamically find the UEN column (first column containing 'UEN')
uen_col = [c for c in master_db_df.columns if "UEN" in c][0]
master_db_df["UEN"] = master_db_df[uen_col].apply(clean_uen)
master_db_df = master_db_df.drop(columns=[uen_col])

# Rename other columns consistently
rename_map = {
    "BRAND_DEAL_NAME_BUSINESS_NAME": "BRAND_NAME",
    "PRIMARY_SSIC_CODE": "SSIC_CODE",
    "ACRA_REGISTERED_NAME": "ACRA_REGISTERED_NAME"
}
master_db_df = master_db_df.rename(columns={k: v for k, v in rename_map.items() if k in master_db_df.columns})

# Clean text columns
for col in ["ACRA_REGISTERED_NAME", "BRAND_NAME"]:
    if col in master_db_df.columns:
        master_db_df[col] = master_db_df[col].apply(clean_text)

# Convert SSIC_CODE to integer if exists
if "SSIC_CODE" in master_db_df.columns:
    master_db_df["SSIC_CODE"] = master_db_df["SSIC_CODE"].astype("Int64")

# Keep only required columns if they exist
required_cols = ["UEN", "ACRA_REGISTERED_NAME", "BRAND_NAME", "SSIC_CODE"]
master_db_df = master_db_df[[c for c in required_cols if c in master_db_df.columns]]

# Filter out rows with missing or empty UEN
master_db_df = master_db_df[master_db_df["UEN"].notna() & (master_db_df["UEN"].str.strip() != "")]

master_db_df


,UEN,ACRA_REGISTERED_NAME,BRAND_NAME,SSIC_CODE
0,04799400B,AIK BEE TEXTILE CO,AIK BEE TEXTILE CO,46411
1,03376200K,SERANGOON GARDEN CLINIC AND DISPENSARY,GARDEN CLINIC,550263
2,06239600E,SALON DE BENZIMEN,SALON DE BENZIMEN,96021
3,06952000C,SU LAN LADIES FASHION,SU LAN LADIES FASHION,14103
4,10381600C,SIN HAI PRINTING SERVICE,SIN HAI PRINTING SERVICE,18113
...,...,...,...,...
7444,201734006N,MISTER MOBILE HOUGANG PTE. LTD.,MISTER MOBILE (HOUGANG),95120
7445,202210879W,MISTER MOBILE CHINATOWN PTE. LTD.,MISTER MOBILE (CHINATOWN),47411
7446,202205507G,MISTER MOBILE PTE. LTD.,MISTER MOBILE HQ,64202
7454,53473046M,BLOONIES,BLOONIES,47742


### Getting ACRA Data (Filter by Live, Live Company only & non relevant ssic code)

In [56]:

folder_path = "Acra_Data"

# Get all CSV file paths inside the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and combine all CSVs
# Using low_memory=False to avoid DtypeWarning for mixed types
df = pd.concat((pd.read_csv(f, low_memory=False) for f in csv_files), ignore_index=True)


df.columns = df.columns.str.upper()


acra_data = df[[
    "UEN",
    "ENTITY_NAME",
    "BUSINESS_CONSTITUTION_DESCRIPTION",
    "ENTITY_TYPE_DESCRIPTION",
    "ENTITY_STATUS_DESCRIPTION",
    "REGISTRATION_INCORPORATION_DATE",
    "PRIMARY_SSIC_CODE",
    "SECONDARY_SSIC_CODE",
    "STREET_NAME",
    "POSTAL_CODE"
]].copy()

# Convert to proper data types
acra_data['UEN'] = acra_data['UEN'].astype('string')
acra_data['ENTITY_NAME'] = acra_data['ENTITY_NAME'].astype('string')
acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'] = acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'].astype('string')
acra_data['ENTITY_TYPE_DESCRIPTION'] = acra_data['ENTITY_TYPE_DESCRIPTION'].astype('string')
acra_data['ENTITY_STATUS_DESCRIPTION'] = acra_data['ENTITY_STATUS_DESCRIPTION'].astype('string')
acra_data['REGISTRATION_INCORPORATION_DATE'] = pd.to_datetime(acra_data['REGISTRATION_INCORPORATION_DATE'], errors='coerce')

# Clean string columns — trim, remove extra spaces, uppercase
for col in [
    'UEN',
    'ENTITY_NAME',
    'BUSINESS_CONSTITUTION_DESCRIPTION',
    'ENTITY_TYPE_DESCRIPTION',
    'ENTITY_STATUS_DESCRIPTION',
    'STREET_NAME',
    'POSTAL_CODE'
]:
    acra_data[col] = (
        acra_data[col]
        .fillna('')
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)
        .str.upper()
    )

# Replace placeholders with NaN for standardization
acra_data.replace(['NA', 'N/A', '-', ''], np.nan, inplace=True)

# Convert registration date to dd-mm-yyyy string (optional)
acra_data['REGISTRATION_INCORPORATION_DATE'] = acra_data['REGISTRATION_INCORPORATION_DATE'].dt.strftime('%d-%m-%Y')

# Filter only live entities (LIVE COMPANY or LIVE)
acra_data = acra_data[
    acra_data['ENTITY_STATUS_DESCRIPTION'].isin(['LIVE COMPANY', 'LIVE'])
].reset_index(drop=True)

# Exclude specific PRIMARY_SSIC_CODE values (supposedly the data would be 600k plus but when we exclude this would lessen)
exclude_codes = [
    46900, 47719, 47749, 47539, 47536, 56123,
    10711, 10712, 10719, 10732, 10733, 93209
]

acra_data = acra_data[~acra_data['PRIMARY_SSIC_CODE'].isin(exclude_codes)].reset_index(drop=True)

In [57]:
acra_data

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,na,FISHERY PORT ROAD,619742
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,20234,SIMS AVENUE,387509
2,00733000J,AIK CHE HIONG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,02-11-1974,32909,46900,ANG MO KIO INDUSTRIAL PARK 2A,568049
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,66126,JELLICOE ROAD,208767
4,01173000E,ANG TECK MOH DEPARTMENT STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,30-10-1974,47711,47214,WOODLANDS STREET 12,738623
...,...,...,...,...,...,...,...,...,...,...
537323,T25LL0518K,ZEUS BARBERS LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,16-05-2025,96021,na,KELANTAN LANE,200031
537324,T25LL0858C,ZENSE SPACE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,01-08-2025,43301,46900,YISHUN INDUSTRIAL STREET 1,768161
537325,T25LL0870A,ZIQZEQ PROCUREMENT LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,04-08-2025,70209,46100,SIN MING LANE,573969
537326,T25LL1049B,ZHONG XIN TRAVEL LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,08-09-2025,79102,79101,JALAN BAHAGIA,320034


### Getting SSIC Industry code

In [58]:
# --- CONFIG ---
file_path = "./SSIC_Code/mapped_ssic_code.xlsx"

# --- LOAD DATA ---
mapped_ssic_code = pd.read_excel(file_path)

# --- STANDARDIZE COLUMN NAMES ---
# Uppercase, strip spaces, replace spaces with underscores
mapped_ssic_code.columns = (
    mapped_ssic_code.columns
    .str.strip()
    .str.upper()
    .str.replace(" ", "_")
)

# --- KEEP ONLY DESIRED COLUMNS ---
columns_to_keep = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "SSIC_CODES", "DESCRIPTION"]
mapped_ssic_code = mapped_ssic_code[columns_to_keep].copy()

# --- CLEAN SSIC_CODES COLUMN ---
mapped_ssic_code["SSIC_CODES"] = (
    pd.to_numeric(mapped_ssic_code["SSIC_CODES"], errors="coerce")  # safely convert to numeric
    .fillna(0)
    .astype(int)
)

# --- CLEAN TEXT COLUMNS ---
text_cols = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "DESCRIPTION"]
mapped_ssic_code[text_cols] = mapped_ssic_code[text_cols].apply(
    lambda col: col.astype(str).str.strip().str.title()
)

# --- REMOVE DUPLICATES & RESET INDEX ---
mapped_ssic_code = mapped_ssic_code.drop_duplicates().reset_index(drop=True)

mapped_ssic_code.head()


,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,SSIC_CODES,DESCRIPTION
0,Retail,Retail,Fashion & Apparel,47711,Retail Sale Of Clothing For Adults
1,Retail,Retail,Fashion & Apparel,47712,Retail Sale Of Children And Infants' Clothing
2,Retail,Retail,Fashion & Apparel,47715,Retail Sale Of Sewing And Clothing Accessories
3,Retail,Retail,Fashion & Apparel,47719,"Retail Sale Of Clothing, Footwear And Leather ..."
4,Retail,Retail,Fashion & Apparel,47510,Retail Sale Of Textiles


### Merge ACRA data with SSIC code

In [59]:
# Convert PRIMARY_SSIC_CODE to int
acra_data["PRIMARY_SSIC_CODE"] = (
    pd.to_numeric(acra_data["PRIMARY_SSIC_CODE"], errors="coerce")
    .fillna(0)
    .astype(int)
)

# Merge based on SSIC code
acra_data_filtered = acra_data.merge(
    mapped_ssic_code,
    how="left",
    left_on="PRIMARY_SSIC_CODE",
    right_on="SSIC_CODES"
)

# Optional: drop the duplicate 'SSIC CODES' column (keep only PRIMARY_SSIC_CODE)
acra_data_filtered = acra_data_filtered.drop(columns=["SSIC_CODES"], errors="ignore")


In [60]:
acra_data_filtered

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,na,FISHERY PORT ROAD,619742,Others,Wholesale Trade,"Food, Beverages & Tobacco","Wholesale Of Livestock, Meat, Poultry, Eggs An..."
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,20234,SIMS AVENUE,387509,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
2,00733000J,AIK CHE HIONG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,02-11-1974,32909,46900,ANG MO KIO INDUSTRIAL PARK 2A,568049,Others,Manufacturing,Other Specialised Manufacturing & Distribution,Other Manufacturing Industries N.E.C.
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,66126,JELLICOE ROAD,208767,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
4,01173000E,ANG TECK MOH DEPARTMENT STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,30-10-1974,47711,47214,WOODLANDS STREET 12,738623,Retail,Retail,Fashion & Apparel,Retail Sale Of Clothing For Adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537323,T25LL0518K,ZEUS BARBERS LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,16-05-2025,96021,na,KELANTAN LANE,200031,Services,Services,Hair Salons & Barbershops,Hairdressing Salons/Shops
537324,T25LL0858C,ZENSE SPACE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,01-08-2025,43301,46900,YISHUN INDUSTRIAL STREET 1,768161,Others,Built Environment & Infrastructure,Construction,Renovation Contractors
537325,T25LL0870A,ZIQZEQ PROCUREMENT LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,04-08-2025,70209,46100,SIN MING LANE,573969,Others,"Finance, Legal & Real Estate","Legal, Accounting & Consultancy Activities",Management Consultancy Services N.E.C.
537326,T25LL1049B,ZHONG XIN TRAVEL LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,08-09-2025,79102,79101,JALAN BAHAGIA,320034,Others,"Tourism, Agency",Travel Agencies & Tour Operators,Travel Agencies And Tour Operators (Mainly Out...


### FIlter Acra data with Master DB to get list of companies havent been researched  by MR

In [61]:

# Ensure both UEN columns are strings for accurate matching
acra_data_filtered['UEN'] = acra_data_filtered['UEN'].astype(str).str.strip().str.upper()
master_db_df['UEN'] = master_db_df['UEN'].astype(str).str.strip().str.upper()

# Filter out rows in acra_data_filtered whose UEN is already in master_db_df
acra_data_filtered = acra_data_filtered[~acra_data_filtered['UEN'].isin(master_db_df['UEN'])]

acra_data_filtered.shape

(533824, 14)

### Filter by  Industry

In [62]:
# wholesale data

ssic_codes = [
    "46", "461", "4610", "46100", "462", "4621", "46211", "46212", "46213", "46219",
    "4622", "46221", "46222", "46223", "46224", "46225", "46229", "463", "4630", "46301",
    "46302", "46303", "46304", "46305", "46306", "46307", "46308", "46309", "464", "4641",
    "46411", "46412", "46413", "46414", "46415", "46416", "4642", "46421", "46422", "46423",
    "46424", "46429", "4643", "46431", "46432", "46433", "46434", "46435", "46436", "46439",
    "4644", "46441", "46442", "46443", "46444", "46445", "46449", "4645", "46451", "46452",
    "46453", "46459", "4646", "46461", "46462", "4647", "46471", "46472", "46473", "46474",
    "46479", "4649", "46491", "46492", "46499", "465", "4651", "46511", "46512", "46513",
    "46514", "4652", "46521", "46522", "46523", "4653", "46530", "4654", "46541", "46542",
    "46543", "46544", "46549", "4655", "46551", "46552", "46559", "4656", "46561", "46562",
    "46563", "4659", "46591", "46592", "46593", "46594", "46595", "46599", "466", "4661",
    "46610", "4662", "46620", "4663", "46631", "46632", "46633", "46634", "46635", "46639",
    "4664", "46641", "46642", "46643", "46649", "4665", "46651", "46659", "4666", "46661",
    "46662", "469", "4690", "46900"
]


acra_data_filtered_by_industry = acra_data_filtered[
    (
        (acra_data_filtered["ENTITY_STATUS_DESCRIPTION"].str.lower() == "live") |
        (acra_data_filtered["ENTITY_STATUS_DESCRIPTION"].str.lower() == "live company")
    )
    &
    (acra_data_filtered["PRIMARY_SSIC_CODE"].astype(str).isin(ssic_codes))
]


acra_data_filtered_by_industry

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,na,FISHERY PORT ROAD,619742,Others,Wholesale Trade,"Food, Beverages & Tobacco","Wholesale Of Livestock, Meat, Poultry, Eggs An..."
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,20234,SIMS AVENUE,387509,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,66126,JELLICOE ROAD,208767,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
12,04129500E,AIK HOE & CO,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,23-01-1975,46551,na,KELANTAN ROAD,200028,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Marine Equipment And Accessories
14,04545400X,AIK HUAT AND COMPANY,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,17-01-1975,46441,25119,KAKI BUKIT AVENUE 1,417943,Others,Wholesale Trade,Household Goods,Wholesale Of Sporting Goods And Equipment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537268,T17LP0162L,ZYA HOLDINGS LIMITED PARTNERSHIP,<NA>,LIMITED PARTNERSHIP,LIVE,21-10-2017,46100,46643,NATHAN ROAD,248728,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale On A Fee Or Commission Basis
537298,T22LL0564C,ZEN ENGINEERING & TRADING LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,31-05-2022,46543,41001,TOH GUAN ROAD EAST,608586,Others,Wholesale Trade,"Machinery, Equipment & Supplies","Wholesale Of Lifts, Escalators And Industrial ..."
537302,T23LL0056G,ZECRYNE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,13-01-2023,46301,47211,BUKIT BATOK STREET 25,658881,Others,Wholesale Trade,"Food, Beverages & Tobacco",Wholesale Of Fruits And Vegetables
537313,T24LL0528K,ZOHMH LIMITED LIABILITY PARTNERSHIP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,07-05-2024,46303,na,WOODLANDS AVENUE 4,730844,Others,Wholesale Trade,"Food, Beverages & Tobacco",Wholesale Of A General Line Of Groceries


### Filter with Fresh Leads

In [63]:
# --- Copy to avoid SettingWithCopyWarning ---
acra_data_filtered_wholesale = acra_data_filtered_by_industry.copy()

# --- UPDATE HERE: Remove rows if UEN exists in recordowl_results.xlsx ---
Fresh_Leads_Wholesale = pd.read_excel("Fresh_Leads_WholeSale.xlsx")


if "UEN" in Fresh_Leads_Wholesale.columns and "UEN" in acra_data_filtered_wholesale.columns:
    filtered = acra_data_filtered_wholesale[~acra_data_filtered_wholesale["UEN"].isin(Fresh_Leads_Wholesale["UEN"])]
else:
    raise ValueError("Column 'UEN' not found in one of the dataframes.")

# sample data 
acra_data_filtered_wholesale = filtered.sample(n=10).reset_index(drop=True)

acra_data_filtered_wholesale.head(10)


,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,202041096W,RICHSEA ENTERPRISE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,21-12-2020,46100,50021,SERANGOON ROAD,218108,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale On A Fee Or Commission Basis
1,202502139K,ULTRA GREAT PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,15-01-2025,46643,46900,KING GEORGE'S AVENUE,208577,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale Of Petrochemical Products
2,53005718B,SWEE HUAT TAN KEE TRADING,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,13-10-2003,46301,na,WHOLESALE CENTRE,110013,Others,Wholesale Trade,"Food, Beverages & Tobacco",Wholesale Of Fruits And Vegetables
3,202500762G,PERPETUAL SOURCE PRIVATE LIMITED,<NA>,LOCAL COMPANY,LIVE COMPANY,07-01-2025,46439,na,TANJONG RHU ROAD,436918,Others,Wholesale Trade,Household Goods,"Wholesale Of Furniture, Home Furnishings And O..."
4,202243964H,CC AVIATION SOLUTIONS PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,09-12-2022,46552,30302,CHANGI SOUTH LANE,486127,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Aircraft Equipment And Supplies
5,201931761R,RSZB INTERNATIONAL DEVELOPMENT PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,23-09-2019,46620,na,RAFFLES PLACE,48616,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale Of Metals And Metal Ores (Except Gen...
6,202112075G,A&K MARINE SUPPLIES PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,06-04-2021,46434,na,TUAS VIEW PLACE,637433,Others,Wholesale Trade,Household Goods,"Wholesale Of Crockery, Cutlery And Kitchen Ute..."
7,202537030N,KAPI KAPI BY LUVI PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,21-08-2025,46412,46413,TAMPINES STREET 64,523660,Others,Wholesale Trade,Household Goods,Wholesale Of Adults' Clothing
8,53446695L,J ELITE FOOD,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,19-02-2022,46593,na,TENGAH PARK AVENUE,692318,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Commercial Food Service Equipment
9,33914900X,LAKESIDE FISHERY TRADING,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,05-01-1985,46302,46900,WOODLANDS DRIVE 50,732897,Others,Wholesale Trade,"Food, Beverages & Tobacco","Wholesale Of Livestock, Meat, Poultry, Eggs An..."


In [ ]:

# Process_data_RecordOwl_df = pd.DataFrame({
#     "UEN": ["193600030K"]
# })

In [64]:
Process_data_RecordOwl_df = acra_data_filtered_wholesale.copy()
Process_data_RecordOwl_df.head(5)

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,202041096W,RICHSEA ENTERPRISE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,21-12-2020,46100,50021,SERANGOON ROAD,218108,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale On A Fee Or Commission Basis
1,202502139K,ULTRA GREAT PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,15-01-2025,46643,46900,KING GEORGE'S AVENUE,208577,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale Of Petrochemical Products
2,53005718B,SWEE HUAT TAN KEE TRADING,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,13-10-2003,46301,na,WHOLESALE CENTRE,110013,Others,Wholesale Trade,"Food, Beverages & Tobacco",Wholesale Of Fruits And Vegetables
3,202500762G,PERPETUAL SOURCE PRIVATE LIMITED,<NA>,LOCAL COMPANY,LIVE COMPANY,07-01-2025,46439,na,TANJONG RHU ROAD,436918,Others,Wholesale Trade,Household Goods,"Wholesale Of Furniture, Home Furnishings And O..."
4,202243964H,CC AVIATION SOLUTIONS PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,09-12-2022,46552,30302,CHANGI SOUTH LANE,486127,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Aircraft Equipment And Supplies


In [65]:
Process_data_RecordOwl_df.shape

(10, 14)

### Get Data from RecordOwl 

In [66]:

client = ApifyClient("apify_api_BKqgA7WLcQMD7dugx62KslGgbrxZ2t3NB2gj")

SOCIAL_MEDIA_DOMAINS = [
    "facebook.com", "linkedin.com", "instagram.com", "youtube.com",
    "tiktok.com", "twitter.com", "x.com", "pinterest.com"
]

def fetch_dataset_items_safe(dataset_client, max_retries=5, initial_wait=3):
    """Safely fetch dataset items with multiple retry strategies."""
    dataset_items = []
    
    for attempt in range(max_retries):
        try:
            # Strategy 1: Try using iterate_items() (streaming)
            try:
                dataset_items = list(dataset_client.iterate_items())
                if dataset_items:
                    return dataset_items
            except (HTTPError, ConnectionError, ProtocolError, Exception) as e:
                if attempt < max_retries - 1:
                    wait_time = initial_wait * (2 ** attempt)  # Exponential backoff
                    print(f"  ⚠️ Iteration method failed (attempt {attempt + 1}/{max_retries}), trying direct fetch in {wait_time}s...")
                    time.sleep(wait_time)
                    continue
                else:
                    print(f"  ⚠️ Iteration method failed after all retries, trying direct fetch...")
            
            # Strategy 2: Try using list_items() (direct pagination)
            try:
                offset = 0
                limit = 100
                while True:
                    page = dataset_client.list_items(offset=offset, limit=limit, clean=True)
                    if not page.items:
                        break
                    dataset_items.extend(page.items)
                    if len(page.items) < limit:
                        break
                    offset += limit
                
                if dataset_items:
                    return dataset_items
            except (HTTPError, ConnectionError, ProtocolError, Exception) as e:
                if attempt < max_retries - 1:
                    wait_time = initial_wait * (2 ** attempt)
                    print(f"  ⚠️ Direct fetch failed (attempt {attempt + 1}/{max_retries}), retrying in {wait_time}s...")
                    time.sleep(wait_time)
                    continue
                else:
                    print(f"  ❌ All fetch methods failed: {e}")
                    return []
                    
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = initial_wait * (2 ** attempt)
                print(f"  ⚠️ Unexpected error (attempt {attempt + 1}/{max_retries}), retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"  ❌ Failed after all retries: {e}")
                return []
    
    return dataset_items

def run_apify_with_retry(client, run_input, uen, max_retries=3):
    """Run Apify with exponential backoff on 403 errors AND verify dataset has items."""
    for attempt in range(max_retries):
        try:
            print(f"  📡 Starting Apify run for {uen} (attempt {attempt + 1}/{max_retries})...")
            run = client.actor("apify/puppeteer-scraper").call(run_input=run_input)
            
            print(f"  ⏳ Waiting for run to complete...")
            run_client = client.run(run["id"])
            run_info = run_client.wait_for_finish()
            
            # CRITICAL FIX: Check if run actually scraped pages, not just if it "succeeded"
            if run_info and "status" in run_info:
                status = run_info.get("status")
                
                # Even if status is "SUCCEEDED", verify dataset actually has items
                if status == "SUCCEEDED" and "defaultDatasetId" in run:
                    # Quick check if dataset has any items
                    try:
                        dataset_check = client.dataset(run["defaultDatasetId"])
                        time.sleep(2)  # Brief wait for dataset to be ready
                        test_items = dataset_check.list_items(limit=1, clean=True)
                        
                        if test_items.items and len(test_items.items) > 0:
                            # Dataset has items - true success!
                            print(f"  ✅ Run succeeded with data")
                            return run, None
                        else:
                            # Status says "SUCCEEDED" but dataset is EMPTY - this is a failure!
                            print(f"  ⚠️ Run completed but dataset is empty (likely 403 block)")
                            # Treat as 403 and retry
                            if attempt < max_retries - 1:
                                wait_time = 30 * (2 ** attempt)
                                print(f"  🔄 Retrying in {wait_time}s...")
                                time.sleep(wait_time)
                                continue
                            else:
                                return None, "Dataset empty after all retries (403 blocking)"
                    except Exception as e:
                        print(f"  ⚠️ Could not verify dataset: {e}")
                        # If we can't check dataset, try to use the run anyway
                        return run, None
                
                elif status != "SUCCEEDED":
                    # Check error message for 403
                    error_msg = str(run_info)
                    if "403" in error_msg or "blocked" in error_msg.lower():
                        if attempt < max_retries - 1:
                            wait_time = 30 * (2 ** attempt)  # 30s, 60s, 120s
                            print(f"  🚫 Request blocked (403), waiting {wait_time}s before retry...")
                            time.sleep(wait_time)
                            continue
            
            return run, None
            
        except Exception as e:
            error_str = str(e)
            if "403" in error_str or "blocked" in error_str.lower():
                if attempt < max_retries - 1:
                    wait_time = 30 * (2 ** attempt)
                    print(f"  🚫 Request blocked (403), waiting {wait_time}s before retry...")
                    time.sleep(wait_time)
                    continue
            return None, f"Apify call failed: {str(e)}"
    
    return None, "Max retries exceeded due to 403 blocking"

all_results = []

for idx, (i, row) in enumerate(Process_data_RecordOwl_df.iterrows(), 1):
    uen = str(row["UEN"]).strip()
    print(f"\n🔎 Processing {uen} ({idx}/{len(Process_data_RecordOwl_df)})")

    # Build pageFunction with proper escaping and improved error handling
    page_function = f"""
    async function pageFunction(context) {{
        const {{ page, log, request }} = context;
        const uen = "{uen}";
        log.info("Visiting RecordOwl for UEN: " + uen);

        try {{
            // Step 1: Wait for search input
            await page.waitForSelector("input[placeholder='Search company name, industry, or address']", {{ timeout: 30000 }});
            log.info("Search input found");
            
            // Step 2: Type UEN into search box with error handling and navigation protection
            try {{
                // Wait for page to be stable (no navigation happening)
                log.info("Waiting for page to stabilize...");
                await new Promise(r => setTimeout(r, 2000)); // Wait for any auto-navigation to complete
                
                // Wait for input to be present and stable
                await page.waitForSelector("input[placeholder='Search company name, industry, or address']", {{ 
                    timeout: 30000,
                    visible: true 
                }});
                
                // Re-find input right before typing (in case page navigated)
                let input = await page.$("input[placeholder='Search company name, industry, or address']");
                if (!input) {{
                    log.error("Input element not found after wait");
                    return {{ status: 'error', uen, error: 'Input element not found' }};
                }}
                
                // Clear and type with retry logic
                let typed = false;
                for (let attempt = 0; attempt < 3; attempt++) {{
                    try {{
                        // Re-find input on each attempt (in case context was destroyed)
                        input = await page.$("input[placeholder='Search company name, industry, or address']");
                        if (!input) {{
                            throw new Error("Input not found on attempt " + (attempt + 1));
                        }}
                        
                        // Click to focus
                        await input.click({{ clickCount: 3 }});
                        await new Promise(r => setTimeout(r, 300)); // Small delay after click
                        
                        // Clear input first
                        await page.evaluate((selector) => {{
                            const el = document.querySelector(selector);
                            if (el) el.value = '';
                        }}, "input[placeholder='Search company name, industry, or address']");
                        
                        // Type UEN
                        await input.type(uen, {{ delay: 100 }});
                        typed = true;
                        log.info("UEN typed successfully: " + uen);
                        break;
                    }} catch (typeErr) {{
                        if (typeErr.message.includes("Execution context was destroyed") || 
                            typeErr.message.includes("navigation")) {{
                            log.info("Navigation occurred during typing (attempt " + (attempt + 1) + "/3), retrying...");
                            // Wait for page to stabilize after navigation
                            await new Promise(r => setTimeout(r, 2000));
                            // Re-wait for input
                            await page.waitForSelector("input[placeholder='Search company name, industry, or address']", {{ 
                                timeout: 10000,
                                visible: true 
                            }});
                            continue;
                        }} else {{
                            throw typeErr;
                        }}
                    }}
                }}
                
                if (!typed) {{
                    log.error("Failed to type UEN after all retries");
                    return {{ status: 'error', uen, error: 'Failed to type UEN after retries' }};
                }}
                
            }} catch (typeErr) {{
                log.error("Error typing UEN: " + typeErr.message);
                return {{ status: 'error', uen, error: 'Failed to type UEN: ' + typeErr.message }};
            }}

            // Step 3: Submit search with flexible waiting strategy
            try {{
                log.info("Clicking submit button...");
                
                // Click submit button first
                await page.click("button[type='submit']");
                log.info("Submit button clicked");
                
                // Wait for either navigation OR results to appear (more flexible)
                // Strategy: Wait for results to appear, with navigation as optional
                try {{
                    // Option 1: Wait for navigation (if it happens) - non-blocking
                    const navigationPromise = page.waitForNavigation({{ 
                        waitUntil: 'networkidle2', 
                        timeout: 30000 
                    }}).catch(() => {{
                        log.info("Navigation did not occur (may be client-side routing)");
                        return null;
                    }});
                    
                    // Option 2: Wait for results to appear (more reliable)
                    const resultsPromise = page.waitForSelector("a[href*='/company/']", {{ 
                        timeout: 60000 
                    }});
                    
                    // Wait for either navigation or results (whichever happens first)
                    await Promise.race([
                        navigationPromise,
                        resultsPromise
                    ]);
                    
                    // Give page time to stabilize
                    await new Promise(r => setTimeout(r, 2000));
                    log.info("Page stabilized after submit");
                    
                }} catch (waitErr) {{
                    // If both navigation and results wait failed, try one more time for results
                    log.info("Initial wait failed, trying again for results: " + waitErr.message);
                    try {{
                        await page.waitForSelector("a[href*='/company/']", {{ timeout: 30000 }});
                        log.info("Results found on retry");
                    }} catch (retryErr) {{
                        log.info("No company links found after submit, might be not found");
                        return {{ status: 'not_found', uen }};
                    }}
                }}
                
            }} catch (navErr) {{
                log.error("Error during submit: " + navErr.message);
                // Don't fail immediately - try to check if results are already there
                try {{
                    const hasResults = await page.$("a[href*='/company/']");
                    if (hasResults) {{
                        log.info("Results found despite submit error");
                    }} else {{
                        return {{ status: 'error', uen, error: 'Submit failed: ' + navErr.message }};
                    }}
                }} catch (checkErr) {{
                    return {{ status: 'error', uen, error: 'Submit failed: ' + navErr.message }};
                }}
            }}

            // Step 4: Verify search results are present
            log.info("Verifying company links are present...");
            try {{
                // Double-check that results are actually there
                await page.waitForSelector("a[href*='/company/']", {{ timeout: 10000 }});
                log.info("Company links confirmed");
            }} catch (e) {{
                log.info("No company links found, might be not found");
                return {{ status: 'not_found', uen }};
            }}

            // Step 5: Find the correct company link (in a new execution context after navigation)
            let companyLink;
            try {{
                companyLink = await page.evaluate((searchUen) => {{
                    const links = Array.from(document.querySelectorAll("a[href*='/company/']"));
                    
                    // Find link where UEN appears in text or URL
                    const uenUpper = searchUen.toUpperCase();
                    const uenLower = searchUen.toLowerCase();
                    
                    for (const a of links) {{
                        const text = (a.innerText || "").toUpperCase();
                        const href = (a.href || "").toLowerCase();
                        
                        // Check if UEN appears in text or URL (case-insensitive)
                        if (text.includes(uenUpper) || href.includes(uenLower)) {{
                            console.log("Found UEN match: " + a.href);
                            return a.href;
                        }}
                    }}
                    
                    // Fallback: Take first company link if available
                    if (links.length > 0) {{
                        console.log("No exact UEN match, using first link: " + links[0].href);
                        return links[0].href;
                    }}
                    
                    console.log("No company links found");
                    return null;
                }}, uen);
                
                if (!companyLink) {{
                    log.info("No company links found on results page");
                    return {{ status: 'not_found', uen }};
                }}
                log.info("Found company link: " + companyLink);
            }} catch (evalErr) {{
                log.error("Error finding company link: " + evalErr.message);
                return {{ status: 'error', uen, error: 'Failed to find company link: ' + evalErr.message }};
            }}

            // Step 6: Navigate to company page if not already there
            if (page.url() !== companyLink) {{
                try {{
                    log.info("Navigating to company page...");
                    await page.goto(companyLink, {{ 
                        waitUntil: 'networkidle2', 
                        timeout: 60000 
                    }});
                    log.info("Company page loaded");
                    
                    // Critical: Wait for page to fully stabilize
                    await new Promise(r => setTimeout(r, 5000));
                }} catch (gotoErr) {{
                    log.error("Error navigating to company page: " + gotoErr.message);
                    return {{ status: 'error', uen, error: 'Failed to load company page: ' + gotoErr.message }};
                }}
            }}

            // Step 7: Wait for content to load (with multiple fallback strategies)
            log.info("Waiting for page content...");
            try {{
                await Promise.race([
                    page.waitForSelector('dt', {{ timeout: 15000 }}),
                    page.waitForSelector('dl', {{ timeout: 15000 }}),
                    page.waitForSelector('.max-w-7xl', {{ timeout: 15000 }}),
                    new Promise(r => setTimeout(r, 10000)) // Fallback: just wait 10s
                ]);
                log.info("Content loaded");
            }} catch (contentErr) {{
                log.info("Content wait timeout, but continuing: " + contentErr.message);
            }}
            
            // Additional stabilization wait
            await new Promise(r => setTimeout(r, 3000));
            
            // Step 7.5: VERIFY we're on the correct company page
            log.info("Verifying UEN on company page...");
            try {{
                const pageUEN = await page.evaluate((searchUen) => {{
                    const pageText = (document.body.innerText || "").toUpperCase();
                    return pageText.includes(searchUen.toUpperCase());
                }}, uen);
                
                if (pageUEN) {{
                    log.info("✓ UEN verified on page: " + uen);
                }} else {{
                    log.info("⚠ Warning: UEN not found in page text, but continuing...");
                }}
            }} catch (verifyErr) {{
                log.info("Could not verify UEN, but continuing: " + verifyErr.message);
            }}
            
            // Step 8: Extract content (in stable context) - ONLY VISIBLE ELEMENTS
            let html_content, title, url;
            try {{
                // Get only the visible HTML content by removing hidden elements
                await page.evaluate(() => {{
                    // Remove all elements that are hidden from view
                    const allElements = document.querySelectorAll('*');
                    allElements.forEach(el => {{
                        const style = window.getComputedStyle(el);
                        // Mark hidden elements with a special attribute
                        if (style.display === 'none' || 
                            style.visibility === 'hidden' || 
                            style.opacity === '0' ||
                            el.hidden ||
                            el.hasAttribute('hidden')) {{
                            el.setAttribute('data-hidden-element', 'true');
                        }}
                    }});
                }});
                
                html_content = await page.content();
                title = await page.title();
                url = page.url();
                log.info("Successfully extracted HTML content (" + html_content.length + " chars)");
            }} catch (extractErr) {{
                log.error("Error extracting content: " + extractErr.message);
                return {{ status: 'error', uen, error: 'Failed to extract content: ' + extractErr.message }};
            }}

            return {{ status: 'success', uen, url, title, html_content }};
            
        }} catch (err) {{
            log.error("Unexpected error in pageFunction: " + err.message);
            log.error("Stack: " + err.stack);
            return {{ status: 'error', uen, error: err.message }};
        }}
    }}
    """

    run_input = {
        "startUrls": [{"url": "https://recordowl.com/"}],
        "useChrome": True,
        "headless": True,
        "stealth": True,
        "pageFunction": page_function,
        "ignoreSslErrors": False,
        "ignoreCorsAndCsp": False,
        "maxRequestRetries": 3,  # Increased retry attempts
        "maxRequestsPerCrawl": 1,  # One page per run
        "maxConcurrency": 1,  # No parallel requests
        "pageLoadTimeoutSecs": 90,  # Optimized timeout
        "pageFunctionTimeoutSecs": 180,  # 3 minutes for pageFunction
        "waitUntil": ["networkidle2"],  # Wait for network to be idle
        # OPTIMIZED: Residential proxies with recommended rotation
        "proxyConfiguration": {
            "useApifyProxy": True,
            "apifyProxyGroups": ["RESIDENTIAL"],  # Residential IPs less likely to be blocked
        },
        "proxyRotation": "RECOMMENDED",  # Optimal proxy rotation strategy
    }

    # Use retry logic for 403 errors (5 attempts = more chances to recover)
    run, error = run_apify_with_retry(client, run_input, uen, max_retries=5)

    if error or not run:
        print(f"  ❌ Apify call failed for {uen}: {error}")
        all_results.append({
            "UEN": uen,
            "Emails": None,
            "Phones": None,
            "Website": None,
            "Facebook": None,
            "LinkedIn": None,
            "Instagram": None,
            "TikTok": None,
            "RecordOwl_Link": None,
            "Error": error or "No run returned"
        })
        time.sleep(10)  # Longer sleep after failure
        continue

    if not run or "defaultDatasetId" not in run:
        print(f"  ⚠️ No valid dataset returned for {uen}")
        all_results.append({
            "UEN": uen,
            "Emails": None,
            "Phones": None,
            "Website": None,
            "Facebook": None,
            "LinkedIn": None,
            "Instagram": None,
            "TikTok": None,
            "RecordOwl_Link": None,
            "Error": "No dataset returned"
        })
        continue

    # Wait for dataset to be ready with progressive checking
    print(f"  ⏳ Waiting for dataset to be ready...")
    time.sleep(5)  # Initial wait
    
    # Try to fetch dataset with progressive waits
    dataset_client = client.dataset(run["defaultDatasetId"])
    for check_attempt in range(3):
        try:
            # Quick check if dataset has items
            test_fetch = dataset_client.list_items(limit=1, clean=True)
            if test_fetch.items:
                break
        except:
            pass
        
        if check_attempt < 2:
            additional_wait = 3 * (check_attempt + 1)
            print(f"  ⏳ Dataset not ready, waiting {additional_wait}s more...")
            time.sleep(additional_wait)
    
    scraped_html, record_owl_url = None, None
    
    # Fetch dataset items with improved error handling
    dataset_items = fetch_dataset_items_safe(
        dataset_client,
        max_retries=5,
        initial_wait=5  # Increased from 3 to 5
    )
    
    # Process items
    if not dataset_items:
        print(f"  ⚠️ Dataset is empty - no items returned!")
    else:
        print(f"  📊 Dataset has {len(dataset_items)} item(s)")
    
    for item in dataset_items:
        if item.get("status") == "success":
            scraped_html = item.get("html_content", "")
            record_owl_url = item.get("url")
            if scraped_html:
                print(f"  ✅ Successfully scraped {uen} ({len(scraped_html)} chars of HTML)")
            else:
                print(f"  ⚠️ Status is 'success' but html_content is empty for {uen}")
        elif item.get("status") == "not_found":
            print(f"  ⚠️ Company not found for UEN {uen}")
        elif item.get("status") == "error":
            print(f"  ❌ Error for {uen}: {item.get('error')}")
        else:
            print(f"  ⚠️ Unknown item status for {uen}: {item.get('status')}")
            print(f"  📋 Item keys: {list(item.keys())}")

    if not scraped_html:
        # Determine the specific reason for failure
        if not dataset_items:
            error_reason = "Dataset empty (likely 403 block at Apify level)"
            print(f"  ❌ {error_reason}")
        elif any(item.get("status") == "not_found" for item in dataset_items):
            error_reason = "Company not found on RecordOwl"
            print(f"  ❌ {error_reason}")
        elif any(item.get("status") == "error" for item in dataset_items):
            error_details = [item.get("error", "Unknown") for item in dataset_items if item.get("status") == "error"]
            error_reason = f"Scraping error: {error_details[0] if error_details else 'Unknown'}"
            print(f"  ❌ {error_reason}")
        else:
            error_reason = "No HTML content retrieved (unknown reason)"
            print(f"  ⚠️ {error_reason}")
            # Debug: show what's in dataset items
            if dataset_items:
                print(f"  🔍 DEBUG - First item: {dataset_items[0]}")
        
        all_results.append({
            "UEN": uen,
            "Emails": None,
            "Phones": None,
            "Website": None,
            "Facebook": None,
            "LinkedIn": None,
            "Instagram": None,
            "TikTok": None,
            "RecordOwl_Link": record_owl_url or None,
            "Error": error_reason
        })
        time.sleep(5)
        continue

    # Parse HTML
    try:
        soup = BeautifulSoup(scraped_html, "html.parser")
        
        # ========== CLEAN HTML: REMOVE HIDDEN/UNWANTED ELEMENTS ==========
        # Remove hidden elements
        for elem in soup.find_all(attrs={"data-hidden-element": "true"}):
            elem.decompose()
        
        # Target company overview (exclude officer/director personal data)
        overview_tab = (soup.select_one("#overview") or 
                       soup.select_one("[aria-labelledby*='overview']") or
                       soup.select_one("div[role='tabpanel']"))
        
        if overview_tab:
            parent = overview_tab
        else:
            parent = soup.select_one("div.max-w-7xl.mx-auto.lg\\:py-6.sm\\:px-6.lg\\:px-8")
            if parent:
                # Remove officer/shareholder sections
                for unwanted in parent.select("#officers, #shareholders, #appointments, "
                                             "[id*='officer'], [id*='shareholder'], [id*='appointment']"):
                    unwanted.decompose()
        
        # Remove non-visible content
        if parent:
            for unwanted in parent.select("script, style, noscript, [style*='display:none']"):
                unwanted.decompose()
        # ========== END CLEAN HTML ==========

        emails, phones, website = [], [], None
        facebook_links, linkedin_links, instagram_links, tiktok_links = [], [], [], []
        
        # Helper function to check if element is visible
        def is_element_visible(element):
            """Check if a BeautifulSoup element appears to be visible (not hidden)."""
            if element is None:
                return False
            # Check for hidden attribute
            if element.has_attr('data-hidden-element'):
                return False
            # Check for common hidden styles
            style = element.get('style', '')
            if any(hidden_style in style.lower() for hidden_style in ['display:none', 'display: none', 'visibility:hidden', 'visibility: hidden']):
                return False
            # Check for hidden/aria-hidden attributes
            if element.get('hidden') or element.get('aria-hidden') == 'true':
                return False
            return True

        if parent:
            # Extract emails
            for a in parent.select("a[href^=mailto]"):
                email = a.get("href", "").replace("mailto:", "").strip()
                if email and email not in emails and "@" in email:
                    emails.append(email)

            # ========== COMPREHENSIVE SINGAPORE PHONE EXTRACTION ==========
            # 
            # ✅ HANDLES ALL POSSIBLE SINGAPORE PHONE NUMBER FORMATS:
            # 
            #    International formats with country code +65:
            #      • +65 6694 5996       (standard international)
            #      • +65-6694-5996       (with dashes)
            #      • +65.6694.5996       (with dots)
            #      • +656694 5996        (partial spacing)
            #      • +6566945996         (no spacing)
            #      • +65 66945996        (no spacing in local part)
            #    
            #    With parentheses:
            #      • (+65) 6694 5996     (parentheses with plus)
            #      • (65) 6694 5996      (parentheses without plus)
            #      • +65(6694)5996       (area code style)
            #      • +65 (6694) 5996     (with spaces)
            #    
            #    Local formats without country code:
            #      • 6566945996          (country code without plus, no spaces)
            #      • 65 6694 5996        (country code with spaces)
            #      • 65-6694-5996        (country code with dashes)
            #      • 6694 5996           (8 digits with space)
            #      • 66945996            (8 digits no space)
            #      • 6694-5996           (8 digits with dash)
            #      • 6694.5996           (8 digits with dot)
            #      • 669 45996           (odd spacing patterns)
            #    
            #    Any combination of separators (spaces, dashes, dots, parentheses, slashes)
            # 
            # ✅ VALIDATION RULES:
            #    • Mobile numbers: Start with 8 or 9 (e.g., 8123 4567, 9123 4567)
            #    • Fixed line: Start with 6 (e.g., 6123 4567, 6694 5996)
            #    • Length: Exactly 8 digits (local) or 10 digits (with country code 65)
            #    • Country code: Singapore +65 only
            #    • Dynamically rejects 20+ non-Singapore country codes
            #    • Filters visible elements only (no hidden HTML data)
            #    • Excludes personal contacts (officers, directors, shareholders)
            # 
            # ❌ AUTOMATICALLY REJECTED:
            #    • Non-Singapore country codes: +60 (Malaysia), +62 (Indonesia), +63 (Philippines),
            #      +66 (Thailand), +81 (Japan), +82 (Korea), +84 (Vietnam), +86 (China), +91 (India), etc.
            #    • Numbers with < 8 or incorrect digit count
            #    • Numbers not starting with 6, 8, or 9 (after country code)
            #    • Hidden or non-visible HTML elements
            #    • Personal/officer contact information
            # 
            # 📤 OUTPUT FORMAT:
            #    All valid numbers are normalized to: +6512345678 (international format)
            
            def validate_sg_phone(digits_str):
                """
                Validate and format Singapore phone number from digit-only string.
                
                This function accepts a string with all separators already removed (only digits)
                and validates it against Singapore phone number rules. It dynamically rejects
                numbers from other countries and only accepts valid Singapore formats.
                
                Args:
                    digits_str (str): String containing only digits (e.g., "6566945996", "66945996")
                
                Returns:
                    str or None: Formatted phone number as "+6512345678" if valid, None otherwise
                
                Examples:
                    validate_sg_phone("6566945996")   -> "+6566945996"    (country code + 8 digits)
                    validate_sg_phone("66945996")     -> "+6566945996"    (8 digits, add country code)
                    validate_sg_phone("63378789")     -> "+6563378789"    (8 digits, NOT rejected as +63)
                    validate_sg_phone("60391312823")  -> None             (Malaysia +60, rejected)
                    validate_sg_phone("63123456789")  -> None             (Philippines +63, rejected)
                    validate_sg_phone("81234567")     -> "+6581234567"    (mobile number)
                """
                if not digits_str or len(digits_str) < 8:
                    return None
                
                # ========== DYNAMIC NON-SINGAPORE COUNTRY CODE BLACKLIST ==========
                # Comprehensive list of international country codes that are NOT Singapore
                # This prevents false positives from extracting parts of non-SG numbers
                non_sg_codes = [
                    # ASEAN Countries
                    "60",   # Malaysia
                    "62",   # Indonesia
                    "63",   # Philippines
                    "66",   # Thailand
                    "84",   # Vietnam
                    "95",   # Myanmar
                    "855",  # Cambodia
                    "856",  # Laos
                    "880",  # Bangladesh
                    
                    # East Asia
                    "81",   # Japan
                    "82",   # South Korea
                    "86",   # China
                    "852",  # Hong Kong
                    "853",  # Macau
                    "886",  # Taiwan
                    
                    # South Asia
                    "91",   # India
                    "92",   # Pakistan
                    "93",   # Afghanistan
                    "94",   # Sri Lanka
                    
                    # Oceania
                    "61",   # Australia
                    "64",   # New Zealand
                    
                    # Others
                    "90",   # Turkey
                    "98",   # Iran
                ]
                
                # ========== STEP 1: VALIDATE 8-DIGIT LOCAL FORMAT FIRST ==========
                # CRITICAL FIX: Check 8-digit local numbers BEFORE country code rejection
                # This prevents false positives like "6337 8789" being rejected as Philippines "+63"
                
                # FORMAT 1: Exactly 8 digits starting with 6/8/9 (local format, no country code)
                # Handles: 6694 5996, 66945996, 6337 8789, 6694-5996, 6694.5996, 669 45996, etc.
                # After stripping separators: 66945996, 63378789, etc.
                # Action: Add +65 country code prefix
                # NOTE: Must check this BEFORE country code rejection to avoid false positives
                if len(digits_str) == 8 and digits_str[0] in "689":
                    return "+65" + digits_str
                
                # ========== STEP 2: REJECT NON-SINGAPORE COUNTRY CODES ==========
                # For numbers with 9+ digits, reject if they start with non-SG country code
                # This prevents extraction of parts of foreign numbers like "+60 3-9131 2823"
                # NOTE: We skip this check for 8-digit numbers (handled above) to avoid false positives
                if len(digits_str) >= 9:
                    for code in non_sg_codes:
                        if digits_str.startswith(code):
                            return None  # Not a Singapore number
                
                # ========== STEP 3: VALIDATE 10-DIGIT INTERNATIONAL FORMAT ==========
                
                # FORMAT 2: Exactly 10 digits starting with 65 and third digit is 6/8/9
                # Handles: +65 6694 5996, +6566945996, 6566945996, 65-6694-5996, etc.
                # After stripping separators: 6566945996
                if len(digits_str) == 10 and digits_str.startswith("65") and digits_str[2] in "689":
                    return "+" + digits_str
                    
                # FORMAT 3: More than 10 digits - search for valid SG pattern within string
                # This handles edge cases where phone number might be concatenated with other digits
                # Example: "Contact: 6566945996 or email" -> digits: "6566945996" 
                elif len(digits_str) > 10:
                    # Search for the pattern "65" followed by valid SG local number (6/8/9...)
                    for i in range(len(digits_str) - 9):
                        if digits_str[i:i+2] == "65" and digits_str[i+2] in "689":
                            # EXTRA VALIDATION: Ensure "65" is not part of another country code
                            # For example, in "865...", the "65" might be part of Cambodia +855
                            if i > 0:
                                # Check if preceding digits form part of a blacklisted code
                                prev_digits = digits_str[max(0, i-2):i]
                                is_part_of_other_code = any(
                                    code.endswith(prev_digits + "65") 
                                    for code in non_sg_codes
                                )
                                if is_part_of_other_code:
                                    continue  # Skip this match, it's part of another country code
                            
                            # Valid Singapore number found
                            return "+" + digits_str[i:i+10]
                
                # ========== STEP 3: REJECT ALL OTHER CASES ==========
                # Don't attempt to force-extract or guess
                # No "last 8 digits" fallback to prevent false positives
                return None
            
            # Method 1: Extract from tel: links (most reliable)
            tel_links = [link for link in parent.select("a[href^='tel:'], a[href^='tel']") 
                        if is_element_visible(link)]
            
            for a in tel_links:
                tel_href = a.get("href", "").replace("tel:", "").strip()
                # Extract digits and validate (validation function handles rejection)
                digits_only = re.sub(r"\D", "", tel_href)
                formatted = validate_sg_phone(digits_only)
                if formatted and formatted not in phones:
                    phones.append(formatted)
            
            # Method 2: Extract from dt/dd structure (company info fields)
            company_keywords = ["company contact", "business contact", "office phone", 
                              "main phone", "business phone", "company phone", "contact number", 
                              "phone", "tel", "mobile", "call", "contact no"]
            exclude_keywords = ["officer", "charge", "employee", "shareholder", "director", 
                              "registration", "person", "individual", "member", "partner",
                              "manager", "owner", "proprietor", "authorized", "representative",
                              "appointment", "designation", "name of", "appointed"]
            
            visible_dt_tags = [dt for dt in parent.select("dt") if is_element_visible(dt)]
            
            for dt in visible_dt_tags:
                dt_text = dt.get_text(strip=True).lower()
                
                # Only extract company-level contacts (not personal)
                is_company = any(kw in dt_text for kw in company_keywords)
                is_personal = any(excl in dt_text for excl in exclude_keywords)
                
                if is_company and not is_personal:
                    dd = dt.find_next_sibling("dd")
                    if dd and is_element_visible(dd):
                        number_text = dd.get_text(" ", strip=True)
                        # Extract digits and validate (validation function handles rejection)
                        all_digits = re.sub(r"\D", "", number_text)
                        formatted = validate_sg_phone(all_digits)
                        if formatted and formatted not in phones:
                            phones.append(formatted)
            
            # Method 3: Fallback text search (only if no phones found via structured data)
            # This method uses comprehensive regex patterns to find phone numbers in free text
            if not phones:
                full_text = parent.get_text()
                
                # COMPREHENSIVE REGEX PATTERNS for all Singapore phone number formats
                # Each pattern handles different separator combinations (spaces, dashes, dots, parentheses)
                sg_patterns = [
                    # Pattern 1: International format with + and any separators
                    # Matches: +65 6694 5996, +65-6694-5996, +65.6694.5996, +656694 5996, +6566945996
                    r"\+[\s\-\.]*65[\s\-\.]*[689][\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d",
                    
                    # Pattern 2: Parentheses format (+65) or (65)
                    # Matches: (+65) 6694 5996, (65) 6694 5996, (+65)66945996, (65)6694-5996
                    r"\([\s\-\.]*\+?[\s\-\.]*65[\s\-\.]*\)[\s\-\.]*[689][\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d",
                    
                    # Pattern 3: Country code in middle with parentheses
                    # Matches: +65(6694)5996, +65 (6694) 5996, 65(6694)5996
                    r"\+?[\s\-\.]*65[\s\-\.]*\([\s\-\.]*[689][\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\)[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d",
                    
                    # Pattern 4: Country code without + (with mandatory separator to avoid false matches)
                    # Matches: 65 6694 5996, 65-6694-5996, 65.6694.5996
                    # Uses negative lookbehind/lookahead to ensure not part of longer number
                    r"(?<!\d)65[\s\-\.]+[689][\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d(?!\d)",
                    
                    # Pattern 5: 8-digit local format (with any separators)
                    # Matches: 6694 5996, 66945996, 6694-5996, 6694.5996, 669 45996
                    # Uses negative lookbehind/lookahead to avoid matching parts of longer numbers
                    r"(?<!\d)[689][\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d[\s\-\.]*\d(?!\d)",
                    
                    # Pattern 6: Country code without separator (edge case)
                    # Matches: 6566945996 (but only if preceded/followed by non-digit)
                    r"(?<!\d)65[689]\d{7}(?!\d)",
                ]
                
                for pattern in sg_patterns:
                    matches = re.findall(pattern, full_text)
                    for match in matches:
                        # Strip all non-digit characters for validation
                        digits = re.sub(r"\D", "", match)
                        # Validate using our comprehensive validation function
                        formatted = validate_sg_phone(digits)
                        if formatted and formatted not in phones:
                            phones.append(formatted)
            # ========== END PHONE EXTRACTION ==========

            # Extract website
            valid_websites = []
            for a in parent.select("a[href^=http]"):
                href = a.get("href", "").strip()
                href_lower = href.lower()
                if not any(domain in href_lower for domain in SOCIAL_MEDIA_DOMAINS):
                    if not any(skip in href_lower for skip in ["recordowl", "apify.com"]):
                        if any(tld in href for tld in [".com", ".sg", ".net", ".org", ".co"]):
                            valid_websites.append(href)
            website = valid_websites[0] if valid_websites else None

        # Extract social media links from entire page
        for a in soup.find_all("a", href=True):
            href = a["href"].strip().lower()
            if "facebook.com" in href and href not in facebook_links:
                facebook_links.append(href)
            elif "linkedin.com" in href and href not in linkedin_links:
                linkedin_links.append(href)
            elif "instagram.com" in href and href not in instagram_links:
                instagram_links.append(href)
            elif "tiktok.com" in href and href not in tiktok_links:
                tiktok_links.append(href)

        all_results.append({
            "UEN": uen,
            "Emails": emails if emails else None,
            "Phones": phones if phones else None,
            "Website": website,
            "Facebook": list(set(facebook_links)) if facebook_links else None,
            "LinkedIn": list(set(linkedin_links)) if linkedin_links else None,
            "Instagram": list(set(instagram_links)) if instagram_links else None,
            "TikTok": list(set(tiktok_links)) if tiktok_links else None,
            "RecordOwl_Link": record_owl_url,
        })
        
        # Print extraction results with actual phone numbers
        if phones:
            phone_list = ", ".join(phones)
            print(f"  ✅ Extracted: {len(emails) if emails else 0} email(s), {len(phones)} phone(s): {phone_list}")
        else:
            print(f"  ✅ Extracted: {len(emails) if emails else 0} email(s), Phone: None found")
        
    except Exception as e:
        print(f"  ❌ Error parsing HTML for {uen}: {e}")
        all_results.append({
            "UEN": uen,
            "Emails": None,
            "Phones": None,
            "Website": None,
            "Facebook": None,
            "LinkedIn": None,
            "Instagram": None,
            "TikTok": None,
            "RecordOwl_Link": record_owl_url or None,
            "Error": f"HTML parsing error: {str(e)}"
        })

    # Dynamic sleep time to avoid rate limiting and 403 blocks
    # Longer delays reduce detection and blocking
    base_sleep = 20  # Increased from 10
    random_addition = (idx % 10) + 5  # 5-14 seconds random
    sleep_time = base_sleep + random_addition  # 25-34 seconds total

    print(f"  💤 Sleeping for {sleep_time}s before next request...")
    time.sleep(sleep_time)

    # Extra delay after every 5th request to further avoid detection
    if idx % 5 == 0:
        extra_wait = 30
        print(f"  🛑 Checkpoint pause: waiting extra {extra_wait}s...")
        time.sleep(extra_wait)

New_Fresh_Leads = pd.DataFrame(all_results)
print("\n✅ Scraping complete!")
print(f"\n📊 Results summary:")
print(f"   Total processed: {len(New_Fresh_Leads)}")
print(f"   With emails: {New_Fresh_Leads['Emails'].notna().sum()}")
print(f"   With phones: {New_Fresh_Leads['Phones'].notna().sum()}")
print(f"   With websites: {New_Fresh_Leads['Website'].notna().sum()}")

New_Fresh_Leads.head(10)


🔎 Processing 202041096W (1/10)
  📡 Starting Apify run for 202041096W (attempt 1/5)...


[apify.puppeteer-scraper runId:MBmBgcDu0KIbgQaAk] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:MBmBgcDu0KIbgQaAk] -> 2025-11-09T06:48:31.503Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:MBmBgcDu0KIbgQaAk] -> 2025-11-09T06:48:31.505Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:MBmBgcDu0KIbgQaAk] -> 2025-11-09T06:48:31.537Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:MBmBgcDu0KIbgQaAk] -> 2025-11-09T06:48:31.684Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:MBmBgcDu0KIbgQaAk] -> 2025-11-09T06:48:33.031Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:MBmBgcDu0KIbgQaAk] -> 2025-11-09T06:48:33.242Z INFO  Configuring Puppeteer Scraper

  ⏳ Waiting for run to complete...
  ✅ Run succeeded with data
  ⏳ Waiting for dataset to be ready...
  📊 Dataset has 1 item(s)
  ✅ Successfully scraped 202041096W (675092 chars of HTML)
  ✅ Extracted: 1 email(s), 1 phone(s): +6598522583
  💤 Sleeping for 26s before next request...

🔎 Processing 202502139K (2/10)
  📡 Starting Apify run for 202502139K (attempt 1/5)...


[apify.puppeteer-scraper runId:IGaR5RfJtXP9dr657] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:IGaR5RfJtXP9dr657] -> 2025-11-09T06:49:44.781Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:IGaR5RfJtXP9dr657] -> 2025-11-09T06:49:44.783Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:IGaR5RfJtXP9dr657] -> 2025-11-09T06:49:44.822Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:IGaR5RfJtXP9dr657] -> 2025-11-09T06:49:45.057Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:IGaR5RfJtXP9dr657] -> 2025-11-09T06:49:45.801Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:IGaR5RfJtXP9dr657] -> 2025-11-09T06:49:45.933Z INFO  Configuring Puppeteer Scraper

  ⏳ Waiting for run to complete...
  ✅ Run succeeded with data
  ⏳ Waiting for dataset to be ready...
  📊 Dataset has 1 item(s)
  ✅ Successfully scraped 202502139K (1422124 chars of HTML)
  ✅ Extracted: 0 email(s), 1 phone(s): +6563333335
  💤 Sleeping for 27s before next request...

🔎 Processing 53005718B (3/10)
  📡 Starting Apify run for 53005718B (attempt 1/5)...


...

[apify.puppeteer-scraper runId:LPmiIKxzNMDQJxjcd] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:LPmiIKxzNMDQJxjcd] -> 2025-11-09T07:00:41.606Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:LPmiIKxzNMDQJxjcd] -> 2025-11-09T07:00:41.609Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:LPmiIKxzNMDQJxjcd] -> 2025-11-09T07:00:41.793Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:LPmiIKxzNMDQJxjcd] -> 2025-11-09T07:00:42.137Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:LPmiIKxzNMDQJxjcd] -> 2025-11-09T07:00:43.690Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:LPmiIKxzNMDQJxjcd] -> 2025-11-09T07:00:43.825Z INFO  Configuring Puppeteer Scraper

  ⏳ Waiting for run to complete...
  ✅ Run succeeded with data
  ⏳ Waiting for dataset to be ready...
  📊 Dataset has 1 item(s)
  ✅ Successfully scraped 53446695L (1125651 chars of HTML)
  ✅ Extracted: 0 email(s), Phone: None found
  💤 Sleeping for 34s before next request...

🔎 Processing 33914900X (10/10)
  📡 Starting Apify run for 33914900X (attempt 1/5)...


[apify.puppeteer-scraper runId:4CLOtZbAesi1n6kVh] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:4CLOtZbAesi1n6kVh] -> 2025-11-09T07:02:11.269Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:4CLOtZbAesi1n6kVh] -> 2025-11-09T07:02:11.271Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:4CLOtZbAesi1n6kVh] -> 2025-11-09T07:02:11.310Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:4CLOtZbAesi1n6kVh] -> 2025-11-09T07:02:11.529Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:4CLOtZbAesi1n6kVh] -> 2025-11-09T07:02:12.433Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:4CLOtZbAesi1n6kVh] -> 2025-11-09T07:02:12.566Z INFO  Configuring Puppeteer Scraper

  ⏳ Waiting for run to complete...
  ✅ Run succeeded with data
  ⏳ Waiting for dataset to be ready...
  📊 Dataset has 1 item(s)
  ✅ Successfully scraped 33914900X (1429584 chars of HTML)
  ✅ Extracted: 0 email(s), 1 phone(s): +6567937781
  💤 Sleeping for 25s before next request...
  🛑 Checkpoint pause: waiting extra 30s...

✅ Scraping complete!

📊 Results summary:
   Total processed: 10
   With emails: 3
   With phones: 6
   With websites: 5


,UEN,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link
0,202041096W,[contactus@airconuncle.com.sg],[+6598522583],https://airconuncle.com.sg,"[https://www.facebook.com/airconunclesg/, http...",None,None,None,https://recordowl.com/company/kitchen-connects...
1,202502139K,None,[+6563333335],https://ultrasupplies.com.sg/,[https://www.facebook.com/ultrasuppliessg],None,[https://www.instagram.com/ultrasupplies/],None,https://recordowl.com/company/ultra-great-pte-ltd
2,53005718B,None,[+6588537339],None,None,None,None,None,https://recordowl.com/company/swee-huat-tan-ke...
3,202500762G,None,None,None,None,None,None,None,https://recordowl.com/company/perpetual-source...
4,202243964H,[contactus@airconuncle.com.sg],[+6598522583],https://airconuncle.com.sg,"[https://www.facebook.com/airconunclesg/, http...",None,None,None,https://recordowl.com/company/kitchen-connects...
5,201931761R,None,None,None,None,None,None,None,https://recordowl.com/company/rszb-internation...
6,202112075G,[contactus@airconuncle.com.sg],[+6598522583],https://airconuncle.com.sg,"[https://www.facebook.com/airconunclesg/, http...",None,None,None,https://recordowl.com/company/kitchen-connects...
7,202537030N,None,None,None,None,None,None,None,https://recordowl.com/company/kapi-kapi-by-luv...
8,53446695L,None,None,https://www.elitefood.sg/,None,None,None,None,https://recordowl.com/company/j-elite-food
9,33914900X,None,[+6567937781],None,[https://www.facebook.com/lakeside.fisherytrad...,None,None,None,https://recordowl.com/company/lakeside-fishery...


In [67]:
New_Fresh_Leads

,UEN,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link
0,202041096W,[contactus@airconuncle.com.sg],[+6598522583],https://airconuncle.com.sg,"[https://www.facebook.com/airconunclesg/, http...",None,None,None,https://recordowl.com/company/kitchen-connects...
1,202502139K,None,[+6563333335],https://ultrasupplies.com.sg/,[https://www.facebook.com/ultrasuppliessg],None,[https://www.instagram.com/ultrasupplies/],None,https://recordowl.com/company/ultra-great-pte-ltd
2,53005718B,None,[+6588537339],None,None,None,None,None,https://recordowl.com/company/swee-huat-tan-ke...
3,202500762G,None,None,None,None,None,None,None,https://recordowl.com/company/perpetual-source...
4,202243964H,[contactus@airconuncle.com.sg],[+6598522583],https://airconuncle.com.sg,"[https://www.facebook.com/airconunclesg/, http...",None,None,None,https://recordowl.com/company/kitchen-connects...
5,201931761R,None,None,None,None,None,None,None,https://recordowl.com/company/rszb-internation...
6,202112075G,[contactus@airconuncle.com.sg],[+6598522583],https://airconuncle.com.sg,"[https://www.facebook.com/airconunclesg/, http...",None,None,None,https://recordowl.com/company/kitchen-connects...
7,202537030N,None,None,None,None,None,None,None,https://recordowl.com/company/kapi-kapi-by-luv...
8,53446695L,None,None,https://www.elitefood.sg/,None,None,None,None,https://recordowl.com/company/j-elite-food
9,33914900X,None,[+6567937781],None,[https://www.facebook.com/lakeside.fisherytrad...,None,None,None,https://recordowl.com/company/lakeside-fishery...


In [68]:
New_Fresh_Leads.to_excel("New_Fresh_Leads_9_12.xlsx", index=False)

### APPEND

In [69]:
import pandas as pd

# Read the two Excel files
fresh_leads_wholesale = pd.read_excel("Fresh_Leads_Wholesale.xlsx")
new_fresh_leads_20 = pd.read_excel("New_Fresh_Leads_9_12.xlsx")

# Display info about each file
print(f"Fresh_Leads_Wholesale: {len(fresh_leads_wholesale)} rows, {len(fresh_leads_wholesale.columns)} columns")
print(f"New_Fresh_Leads_20: {len(new_fresh_leads_20)} rows, {len(new_fresh_leads_20.columns)} columns")

# Append the two dataframes (concatenate vertically)
combined_leads = pd.concat([fresh_leads_wholesale, new_fresh_leads_20], ignore_index=True)

print(f"\n✅ Combined: {len(combined_leads)} total rows")

# Display first few rows
combined_leads.shape

Fresh_Leads_Wholesale: 358 rows, 10 columns
New_Fresh_Leads_20: 10 rows, 9 columns

✅ Combined: 368 total rows


(368, 10)

In [70]:
# Check how many rows have phone numbers (not NaN)
phone_count = combined_leads['Phones'].notna().sum()
total_rows = len(combined_leads)  # Fixed: should be len(combined_leads), not len(phone_count)

print(f"📞 Rows with phone numbers: {phone_count}")
print(f"📊 Total rows: {total_rows}")
print(f"📈 Percentage with phones: {(phone_count / total_rows * 100):.1f}%")

📞 Rows with phone numbers: 105
📊 Total rows: 368
📈 Percentage with phones: 28.5%


In [71]:
# Check if all UEN values are unique
total_uens = len(combined_leads['UEN'])
unique_uens = combined_leads['UEN'].nunique()
duplicate_count = total_uens - unique_uens

print("=" * 50)
print("UEN UNIQUENESS CHECK")
print("=" * 50)

if duplicate_count == 0:
    print(f"✅ All UENs are UNIQUE!")
    print(f"   Total UENs: {total_uens}")
else:
    print(f"⚠️  Found DUPLICATES!")
    print(f"   Total UENs: {total_uens}")
    print(f"   Unique UENs: {unique_uens}")
    print(f"   Duplicate UENs: {duplicate_count}")
    
    # Show which UENs are duplicated
    print(f"\n📋 Duplicate UEN values:")
    duplicated_uens = combined_leads[combined_leads['UEN'].duplicated(keep=False)]['UEN'].value_counts()
    print(duplicated_uens)

UEN UNIQUENESS CHECK
✅ All UENs are UNIQUE!
   Total UENs: 368


In [72]:
# Save combined leads to Excel file
output_filename = "Fresh_Leads_WholeSale.xlsx"

combined_leads.to_excel(output_filename, index=False)

print("=" * 50)
print("✅ FILE SAVED SUCCESSFULLY!")
print("=" * 50)
print(f"📁 Filename: {output_filename}")
print(f"📊 Total rows: {len(combined_leads)}")
print(f"📋 Total columns: {len(combined_leads.columns)}")
print(f"📞 Rows with phones: {combined_leads['Phones'].notna().sum()}")
print(f"📧 Rows with emails: {combined_leads['Emails'].notna().sum()}")

✅ FILE SAVED SUCCESSFULLY!
📁 Filename: Fresh_Leads_WholeSale.xlsx
📊 Total rows: 368
📋 Total columns: 10
📞 Rows with phones: 105
📧 Rows with emails: 66


In [73]:
combined_leads

,UEN,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link,Error
0,201819442H,['enquiry@bell-1.com'],['+6562590183'],https://bell-1.com,['https://www.facebook.com/bell1lubricant/'],NaN,['https://www.instagram.com/bell1lubricant/'],NaN,https://recordowl.com/company/bell-1-singapore...,NaN
1,202534677Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://recordowl.com/company/nuu-nii-pte-ltd,NaN
2,201925464M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://recordowl.com/company/cloud-frontier-a...,NaN
3,202108485W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://recordowl.com/company/golden-garland-c...,NaN
4,202519922K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://recordowl.com/company/xinyuan-shoes-pt...,NaN
...,...,...,...,...,...,...,...,...,...,...
363,201931761R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://recordowl.com/company/rszb-internation...,NaN
364,202112075G,['contactus@airconuncle.com.sg'],['+6598522583'],https://airconuncle.com.sg,"['https://www.facebook.com/airconunclesg/', 'h...",NaN,NaN,NaN,https://recordowl.com/company/kitchen-connects...,NaN
365,202537030N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://recordowl.com/company/kapi-kapi-by-luv...,NaN
366,53446695L,NaN,NaN,https://www.elitefood.sg/,NaN,NaN,NaN,NaN,https://recordowl.com/company/j-elite-food,NaN
